<a href="https://colab.research.google.com/github/ecemdenizb/artificial-intelligence/blob/main/sentencePiece_sentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece

In [ ]:
import sentencepiece as spm
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
!wget https://www.dropbox.com/s/a7z7suwt85llb9a/hepsiburada.csv?dl=1 -O hepsiburada.csv

--2021-03-20 07:15:02--  https://www.dropbox.com/s/a7z7suwt85llb9a/hepsiburada.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/a7z7suwt85llb9a/hepsiburada.csv [following]
--2021-03-20 07:15:02--  https://www.dropbox.com/s/dl/a7z7suwt85llb9a/hepsiburada.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7c899b3a7cf7ed60f713a82bb3.dl.dropboxusercontent.com/cd/0/get/BLBSLAoZmfjI0AClSJpDltYNd-Auq0HGYtaRXOMuVJfzntFsHAFBBC-y2sx-UMCx8v2eFUPhR_m6mp42i3Q_UKQHCbUwiIygWuYJQz1CJdFGRdDX6qv1HVK3Ug46AoPX1tMi-UCUHR8Keq00IFF1Tjyh/file?dl=1# [following]
--2021-03-20 07:15:02--  https://uc7c899b3a7cf7ed60f713a82bb3.dl.dropboxusercontent.com/cd/0/get/BLBSLAoZmfjI0AClSJpDltYNd-Auq0HGYtaRXOMuVJfzntFsHAFBBC-y2sx-UMCx8v2eFUPhR_m6

In [ ]:
vocab_size=5000

In [ ]:
dataset = pd.read_csv('hepsiburada.csv')
dataset.head()

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...


In [ ]:
target = dataset['Rating'].values.tolist()
reviews = dataset['Review']

In [ ]:
reviews.to_csv('reviews.txt', index=False, header=False)
!head reviews.txt

3 yıldır tık demedi. :)
3 yıldır kullanıyorum müthiş 
"Ürün bugün elime geçti çok fazla inceleme fırsatım olmadı ancak gözüme çarpan ilk şey usb bağlantısı için pil kapağının altında çok güzel yer yapmış olmaları, daha önceden o aparatı kaybetmiş biri olarak bu özelliğini çok sevdim. Ayrıca 1000dpi olması çok iyi bu tip farelerin çoğu 800dpi oluyor. Eğer sizde benim gibi masaüstü bilgisayarınızdan oyuncu faresi ile yüksek dpi ayarına alışmış biriyseniz windows ayarından fareyi en hızlıya getirin ve bu fareyi sipariş verin pişman olmazsınız. Teşekkürler Logitech"
Almaya karar verdim. Hemencecik geldi. Keyifle kullanıyorum
Günlük kullanımınızı çok çok iyi karsılıyor kısaca mükemmel
gayet güzel
Çok kaliteli bir ürün ve fiyatı da uygun. Uzun araştırmalarım sonucu aldım ve çok memnunum.
yaklaşık 5 senedir kullanıyorum. defalarca düşmesine rağmen banamısın demedi. 5 yıl daha gider diye düşünüyorum.
Ürün günlük kullanım için çok uygun. Ürünle birlikte 1 adet pil de çıkıyor. İş ve günlük kulla

In [ ]:
reviews=reviews.to_list()
cutoff = int(len(reviews) * 0.8)
x_train, x_test = reviews[:cutoff], reviews[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [ ]:
x_train_tokens = []
x_test_tokens= []

In [ ]:
spm.SentencePieceTrainer.Train(f'--input=reviews.txt --model_prefix=reviews_{vocab_size} --vocab_size={vocab_size} --minloglevel=2')

In [ ]:
!ls

hepsiburada.csv     reviews_5000.vocab	sample_data
reviews_5000.model  reviews.txt


In [ ]:
tokenizer=spm.SentencePieceProcessor()
tokenizer.load('reviews_5000.model')

True

In [ ]:
for review in x_train:
  x_train_tokens.append(tokenizer.EncodeAsIds(review))
for review in x_test:
  x_test_tokens.append(tokenizer.EncodeAsIds(review))


In [ ]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
np.mean(num_tokens), np.max(num_tokens), np.argmax(num_tokens)

(37.468970870277666, 930, 225796)

In [ ]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

117

In [ ]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)
x_train_pad.shape, x_test_pad.shape

((194797, 117), (48700, 117))

In [ ]:
embedding_size = 50
model = Sequential()

model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 117, 50)           250000    
_________________________________________________________________
gru_3 (GRU)                  (None, 117, 16)           3264      
_________________________________________________________________
gru_4 (GRU)                  (None, 117, 8)            624       
_________________________________________________________________
gru_5 (GRU)                  (None, 4)                 168       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 254,061
Trainable params: 254,061
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = Adam(lr=1e-3)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=optimizer,
              metrics=['accuracy'])


In [ ]:
from tensorflow.data import Dataset
train_ds = Dataset.zip((Dataset.from_tensor_slices(x_train_pad),Dataset.from_tensor_slices(y_train)))
train_ds = train_ds.shuffle(1024).batch(256)
val_ds = Dataset.zip((Dataset.from_tensor_slices(x_test_pad), Dataset.from_tensor_slices(y_test)))
val_ds = val_ds.batch(256)
steps_per_epoch = len(x_train_pad) // 256
validation_steps = len(x_test_pad) // 256

In [ ]:
model.fit(train_ds.repeat(), epochs=5, steps_per_epoch=steps_per_epoch, validation_data=val_ds.repeat(), validation_steps=validation_steps)

Epoch 1/5
760/760 [==============================] - 128s 162ms/step - loss: 0.2842 - accuracy: 0.8465 - val_loss: 0.2127 - val_accuracy: 0.9428
Epoch 2/5
760/760 [==============================] - 122s 161ms/step - loss: 0.1637 - accuracy: 0.9532 - val_loss: 0.1465 - val_accuracy: 0.9480
Epoch 3/5
760/760 [==============================] - 124s 164ms/step - loss: 0.1071 - accuracy: 0.9636 - val_loss: 0.1331 - val_accuracy: 0.9516
Epoch 4/5
760/760 [==============================] - 125s 164ms/step - loss: 0.0836 - accuracy: 0.9720 - val_loss: 0.1389 - val_accuracy: 0.9549
Epoch 5/5
760/760 [==============================] - 125s 165ms/step - loss: 0.0685 - accuracy: 0.9783 - val_loss: 0.1523 - val_accuracy: 0.9567
